In [1]:
import pickle
import lightgbm as lgb
import pandas as pd

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/dask/config.py:129: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/dask/dataframe/utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
import pickle
filename="./model.lgb"
lgb_model = pickle.load(open(filename, 'rb'))
lgb_model.params

{'lambda_l1': 4.5465750492375764e-07,
 'lambda_l2': 3.204615863122189e-05,
 'learning_rate': 0.2279641775396567,
 'num_leaves': 126,
 'verbosity': -1,
 'objective': 'binary',
 'metric': 'auc',
 'num_iterations': 1000,
 'early_stopping_round': None}

In [18]:
daily_df = pd.read_csv("HealthTrackerData/dailyActivity_summary.csv")
avg_daily_df = daily_df.groupby(["Id"]).agg({"TotalSteps":"mean"}).rename(columns = {"TotalSteps": "avg_daily_step"}).reset_index()
avg_daily_df["avg_daily_step"] = avg_daily_df.avg_daily_step.astype('int')
avg_daily_df.head()

,Id,avg_daily_step
0,1503960366,12520
1,1624580081,4734
2,1644430081,7282
3,1844505072,4689
4,1927972279,2235


In [19]:
daily_weight_log_df = pd.read_csv("HealthTrackerData/dailyWeightLog_summary.csv")
avg_daily_weight_log_df = daily_weight_log_df.groupby(["Id"]).agg({"BMI":"mean"}).rename(columns = {"BMI": "avg_BMI"}).reset_index()
avg_daily_weight_log_df["avg_BMI"] = avg_daily_weight_log_df.avg_BMI.astype('int')
avg_daily_weight_log_df

,Id,avg_BMI
0,1503960366,22
1,1927972279,47
2,2873212765,21
3,4319703577,27
4,4558609924,27
5,5577150313,28
6,6962181067,24
7,8877689391,25


In [20]:
avg_user_report_df = avg_daily_weight_log_df.merge(avg_daily_df, left_on="Id", right_on="Id", how="inner")
avg_user_report_df

,Id,avg_BMI,avg_daily_step
0,1503960366,22,12520
1,1927972279,47,2235
2,2873212765,21,7555
3,4319703577,27,7769
4,4558609924,27,7685
5,5577150313,28,7243
6,6962181067,24,9794
7,8877689391,25,10656


In [21]:
import numpy as np

n = avg_user_report_df.shape[0]

temp = pd.DataFrame({'smoking_status_formerly smoked': [0]*n, 
     'smoking_status_never smoked': [0]*n,
           'smoking_status_smokes': [0]*n, 
          'smoking_status_unknown': [0]*n, 
           'work_type_children': [0]*n,
           'work_type_govt_job': [0]*n, 
           'work_type_never_worked': [0]*n, 
           'work_type_private': [1]*n,
           'work_type_self-employed': [0]*n,
           'gender': [1]*n, 
           'age': [50]*n, 
           'hypertension': [0]*n,
           'heart_disease': [0]*n, 
           'ever_married': [1]*n, 
           'Residence_type': [1]*n, 
           'avg_glucose_level': [92]*n,
           'bmi': avg_user_report_df.avg_BMI.to_list()})

In [22]:
avg_user_report_df["prob_stroke"] = lgb_model.predict(temp.values)

In [23]:
avg_user_report_df.sort_values(by="prob_stroke", ascending=True).head(20)

,Id,avg_BMI,avg_daily_step,prob_stroke
0,1503960366,22,12520,0.000006
7,8877689391,25,10656,0.000010
2,2873212765,21,7555,0.000035
3,4319703577,27,7769,0.000064
4,4558609924,27,7685,0.000064
6,6962181067,24,9794,0.000089
5,5577150313,28,7243,0.000112
1,1927972279,47,2235,0.002183


In [24]:
def map_cardiovascular_mortality_rate(x):
    if x < 4000:
        return 31.6/1000
    elif 4000 <= x < 7999:
        return 6.6/1000
    elif 8000 <= x < 11999:
        return 2.1/1000
    elif x>=12000:
        return 1/1000
    

avg_user_report_df["mortality_rate_cardic"] = avg_user_report_df.avg_daily_step.apply(map_cardiovascular_mortality_rate)
avg_user_report_df["extra_cardic_mortality__rate"] = avg_user_report_df["prob_stroke"]*avg_user_report_df["mortality_rate_cardic"]*100
avg_user_report_df.sort_values(by="extra_cardic_mortality__rate", ascending=True)

,Id,avg_BMI,avg_daily_step,prob_stroke,mortality_rate_cardic,extra_cardic_mortality__rate
0,1503960366,22,12520,0.000006,0.0010,6.422627e-07
7,8877689391,25,10656,0.000010,0.0021,2.193839e-06
6,6962181067,24,9794,0.000089,0.0021,1.872210e-05
2,2873212765,21,7555,0.000035,0.0066,2.292511e-05
3,4319703577,27,7769,0.000064,0.0066,4.203625e-05
4,4558609924,27,7685,0.000064,0.0066,4.203625e-05
5,5577150313,28,7243,0.000112,0.0066,7.359351e-05
1,1927972279,47,2235,0.002183,0.0316,6.899665e-03
